# **Computation of PPP Metrics for Ensemble Runs**

**Important metrics:**

- SH sea ice extent/volume
- regional sea ice extent/volume
- regional NPP, PCO2, SSS, SST
- gridpoint-level fields for sea ice concentration, SST, SSS, NPP, PCO2
- sea ice edge position (latitude at which sea ice concentration drops below 15%)

In [2]:
import os
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")
warnings.filterwarnings("ignore", message="invalid value encountered in reduce")

import xarray as xr
import numpy as np
import math
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import hmei.processing as pcs
import hmei.visualization as vis

In [4]:
rootdir = '/local/projects/so_predict/esm2m_froelicher/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/'

# Import Control Run Data

In [3]:
## variable names
CN_INV = 'CN_INV'
NPP = 'NPP'
PCO2SURF = 'PCO2SURF'
SIE = 'SIE'
SIV = 'SIV'
SI_EDGE = 'SI_EDGE'
SSS = 'SSS'
SST = 'SST'
MLD ='MLD'

In [4]:
def open_ctrl(var, reg, timescale):
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    subdir = 'CTRL/'
    filename = var.lower()+'_'+reg+'_'+timescale+'_mean.nc'
    return xr.open_dataset(writedir+subdir+var.upper()+'/'+filename)

sst_global_annual_mean  = open_ctrl(SST,'global','annual')
sss_global_annual_mean  = open_ctrl(SSS,'global','annual')
pco2_global_annual_mean = open_ctrl(PCO2SURF,'global','annual')
npp_global_annual_mean  = open_ctrl(NPP,'global','annual')
cn_global_annual_mean   = open_ctrl(CN_INV,'global','annual')
sie_global_annual_mean  = open_ctrl(SIE,'global','annual')
siv_global_annual_mean  = open_ctrl(SIV,'global','annual')
mld_global_annual_mean = open_ctrl(MLD,'global','annual')

sst_global_monthly_mean  = open_ctrl(SST,'global','monthly')
sss_global_monthly_mean  = open_ctrl(SSS,'global','monthly')
pco2_global_monthly_mean = open_ctrl(PCO2SURF,'global','monthly')
npp_global_monthly_mean  = open_ctrl(NPP,'global','monthly')
cn_global_monthly_mean   = open_ctrl(CN_INV,'global','monthly')
sie_global_monthly_mean  = open_ctrl(SIE,'global','monthly')
siv_global_monthly_mean  = open_ctrl(SIV,'global','monthly')
mld_global_monthly_mean = open_ctrl(MLD,'global','monthly')

sst_so_annual_mean  = open_ctrl(SST,'so','annual')
sss_so_annual_mean  = open_ctrl(SSS,'so','annual')
pco2_so_annual_mean = open_ctrl(PCO2SURF,'so','annual')
npp_so_annual_mean  = open_ctrl(NPP,'so','annual')
cn_so_annual_mean   = open_ctrl(CN_INV,'so','annual')
sie_so_annual_mean  = open_ctrl(SIE,'so','annual')
siv_so_annual_mean  = open_ctrl(SIV,'so','annual')
mld_so_annual_mean = open_ctrl(MLD,'so','annual')

sst_so_monthly_mean  = open_ctrl(SST,'so','monthly')
sss_so_monthly_mean  = open_ctrl(SSS,'so','monthly')
pco2_so_monthly_mean = open_ctrl(PCO2SURF,'so','monthly')
npp_so_monthly_mean  = open_ctrl(NPP,'so','monthly')
cn_so_monthly_mean   = open_ctrl(CN_INV,'so','monthly')
sie_so_monthly_mean  = open_ctrl(SIE,'so','monthly')
siv_so_monthly_mean  = open_ctrl(SIV,'so','monthly')
mld_so_monthly_mean = open_ctrl(MLD,'so','monthly')

## Compute Climatology

In [5]:
sst_global_annual_clim = sst_global_annual_mean.mean(dim='year')
sss_global_annual_clim = sss_global_annual_mean.mean(dim='year')
pco2_global_annual_clim = pco2_global_annual_mean.mean(dim='year')
npp_global_annual_clim = npp_global_annual_mean.mean(dim='year')
cn_global_annual_clim = cn_global_annual_mean.mean(dim='year')
sie_global_annual_clim = sie_global_annual_mean.mean(dim='year')
siv_global_annual_clim = siv_global_annual_mean.mean(dim='year')
mld_global_annual_clim = mld_global_annual_mean.mean(dim='year')

sst_so_annual_clim = sst_so_annual_mean.mean(dim='year')
sss_so_annual_clim = sss_so_annual_mean.mean(dim='year')
pco2_so_annual_clim = pco2_so_annual_mean.mean(dim='year')
npp_so_annual_clim = npp_so_annual_mean.mean(dim='year')
cn_so_annual_clim = cn_so_annual_mean.mean(dim='year')
sie_so_annual_clim = sie_so_annual_mean.mean(dim='year')
siv_so_annual_clim = siv_so_annual_mean.mean(dim='year')
mld_so_annual_clim = mld_so_annual_mean.mean(dim='year')

sst_global_monthly_clim = sst_global_monthly_mean.groupby('time.month').mean(dim='time')
sss_global_monthly_clim = sss_global_monthly_mean.groupby('time.month').mean(dim='time')
pco2_global_monthly_clim = pco2_global_monthly_mean.groupby('time.month').mean(dim='time')
npp_global_monthly_clim = npp_global_monthly_mean.groupby('time.month').mean(dim='time')
cn_global_monthly_clim = cn_global_monthly_mean.groupby('time.month').mean(dim='time')
sie_global_monthly_clim = sie_global_monthly_mean.groupby('time.month').mean(dim='time')
siv_global_monthly_clim = siv_global_monthly_mean.groupby('time.month').mean(dim='time')
mld_global_monthly_clim = mld_global_monthly_mean.groupby('time.month').mean(dim='time')

sst_so_monthly_clim = sst_so_monthly_mean.groupby('time.month').mean(dim='time')
sss_so_monthly_clim = sss_so_monthly_mean.groupby('time.month').mean(dim='time')
pco2_so_monthly_clim = pco2_so_monthly_mean.groupby('time.month').mean(dim='time')
npp_so_monthly_clim = npp_so_monthly_mean.groupby('time.month').mean(dim='time')
cn_so_monthly_clim = cn_so_monthly_mean.groupby('time.month').mean(dim='time')
sie_so_monthly_clim = sie_so_monthly_mean.groupby('time.month').mean(dim='time')
siv_so_monthly_clim = siv_so_monthly_mean.groupby('time.month').mean(dim='time')
mld_so_monthly_clim = mld_so_monthly_mean.groupby('time.month').mean(dim='time')

In [6]:
g = xr.concat([sst_so_monthly_clim,sst_so_monthly_clim-10], 'test')

# ANNUAL

In [7]:
subdir = 'SST_ENSEMBLE/'
x = xr.open_dataset(rootdir+'SST_ENSEMBLE/sst_ENS01_neg01_0170_0179.nc')
y = xr.open_dataset(rootdir+'SST_ENSEMBLE/sst_ENS01_pos01_0170_0179.nc')


In [8]:
coords = list(x.coords)
if (coords.count('XT') == 1) and (coords.count('YT') == 1) and (coords.count('TIME') == 1):
    print('a')

if (coords.count('XT_OCEAN') == 1) and (coords.count('YT_OCEAN') == 1) and (coords.count('TIME') == 1):
    print('b')
else:
    print('c')

c


In [9]:
m = x['sst']
m.name = 'test'
m.copy(deep=True)

<xarray.DataArray 'test' (time: 120, yt_ocean: 200, xt_ocean: 360)>
[8640000 values with dtype=float32]
Coordinates:
  * time      (time) object 0170-01-16 12:00:00 ... 0179-12-16 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -81.5 -80.5 -79.5 -78.5 ... 86.5 87.5 88.5 89.5
Attributes:
    long_name:      Potential temperature
    units:          degrees K
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  sea_surface_temperature

In [10]:
## combine 
z = xr.concat([x,y], 'nEns')
# z = z.assign_coords({'nEns':[140,170], 'time':np.arange(1,121)})
# z

## reorder dimensions
# z.transpose('time','nEns','xt_ocean','yt_ocean')

In [ ]:
reg_masks = xr.open_dataset(writedir+'regional_global_masks.nc')
regions = list(reg_masks.data_vars)

ocean_grid = xr.open_dataset(rootdir+'GRID/ocean.static.nc')
geolat_t = ocean_grid.geolat_t
geolon_t = ocean_grid.geolon_t

time = np.arange(1,121)

start_yrs = ['0170_0179', '0022_0031', '0064_0073', '0106_0115', '0232_0241', '0295_0304']

rootdir = '/local/projects/so_predict/esm2m_froelicher/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/'

# var = 'sst'
# subdir = var.upper()+'_ENSEMBLE/'

# variables = np.array([['SST','sst','sst'], ['SSS','sss','sss'], ['MLD','mld','mld'], ['NPP','NPP','NPP'],
#                       ['PCO2SURF','pco2surf','pco2surf'], ['CN','CN_inv','CN_inv'], ['HI','HI_ice','HI']])
variables = np.array([['SST','sst','sst'], ['SSS','sss','sss'], ['MLD','mld','mld'], ['NPP','NPP','NPP'],
                      ['PCO2SURF','pco2surf','pco2surf'], ['CN','CN_inv','CN_inv']])

for var in variables:
    yrs_list = []
    subdir = var[0]+'_ENSEMBLE/'
    
    ## iterate through the start years
    for (i,yrs) in zip(range(1,7), start_yrs):
        ens_list = []
        
        ## iterate through ensembles
        for sign in ['neg', 'pos']:
            for pert in range(1,21):
                filename = var[1]+'_ENS'+f'{i:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'

                ds = xr.open_dataset(rootdir+subdir+filename)

                glob_area = xr.where(np.isnan(ds[var[2]][0]), np.nan, ocean_grid['area_t'])
                    
                ## reassign coord names to ensure continuity between datasets
                coords = list(ds.coords)
                if (coords.count('XT') == 1) and (coords.count('YT') == 1) and (coords.count('TIME') == 1):
                    ds = ds.rename({'XT':'xt_ocean', 'YT':'yt_ocean', 'TIME':'time'})

                if (coords.count('XT_OCEAN') == 1) and (coords.count('YT_OCEAN') == 1) and (coords.count('TIME') == 1):
                    ds = ds.rename({'XT_OCEAN':'xt_ocean', 'YT_OCEAN':'yt_ocean', 'TIME':'time'})
                
                ## assign ocean_grid coords for continuity
                ds = ds.assign_coords({'xt_ocean':ocean_grid.xt_ocean, 'yt_ocean':ocean_grid.yt_ocean})
                
                ## compute timeseries for each region
                reg_list = []
                for reg in regions:
                    area = glob_area.where(reg_masks[reg] == 1)
                    area_sum = area.sum(dim={'xt_ocean', 'yt_ocean'})
                    reg_gridcell = ds[var[2]].where(reg_masks[reg] == 1)
                    
                    ## annual
                    reg_mean = (reg_gridcell * ocean_grid['area_t']).groupby('time.year').mean(dim='time').sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
                    
                    ## monthly
                    # reg_mean = (reg_gridcell * ocean_grid['area_t']).sum(dim={'xt_ocean', 'yt_ocean'}) / area_sum
                    
                    reg_mean.name = reg
                    reg_list.append(reg_mean.copy(deep=True))
                    
                this_ens = xr.merge(reg_list)
                this_ens.attrs['name'] = var[2].lower()+'_so_annual_mean'
                
                ens_list.append(this_ens.copy(deep=True))
        
        these_ens = xr.concat(ens_list, 'nEns')
        yrs_list.append(these_ens)
    
    all_ens = xr.concat(yrs_list, 'nStart')
    
    filename = var[1].lower()+'_ens_so_annual_mean.nc'
    all_ens.to_netcdf(writedir+subdir+filename)

In [8]:
pcs.dir_inspect(writedir)

['CN_INV_ENSEMBLE',
 'CTRL',
 'MLD_ENSEMBLE',
 'NPP_ENSEMBLE',
 'PCO2SURF_ENSEMBLE',
 'SSS_ENSEMBLE',
 'SST_ENSEMBLE',
 'regional_global_masks.nc',
 'regional_so_masks.nc']

In [13]:
pcs.dir_inspect(writedir+'SST_ENSEMBLE')

['sst_ens_so_annual_mean.nc']

In [15]:
xr.open_dataset(writedir+'/SST_ENSEMBLE/sst_ens_so_annual_mean.nc')

<xarray.Dataset>
Dimensions:        (nEns: 40, nStart: 6, year: 60)
Coordinates:
  * year           (year) int64 22 23 24 25 26 27 28 ... 299 300 301 302 303 304
    time           (nStart) object ...
Dimensions without coordinates: nEns, nStart
Data variables:
    SouthernOcean  (nStart, nEns, year) float32 ...
    Weddell        (nStart, nEns, year) float32 ...
    Indian         (nStart, nEns, year) float32 ...
    WestPacific    (nStart, nEns, year) float32 ...
    Ross           (nStart, nEns, year) float32 ...
    AmundBell      (nStart, nEns, year) float32 ...
Attributes:
    name:     sst_so_annual_mean